In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path
import pandas as pd
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Cleaning up the Dataset

In [2]:
path_to_labels="/Users/muhammadwaliji/Desktop/facial_expressions-master/data/legend.csv"

data=pd.read_csv(path_to_labels)


In [3]:
data.head()


,user.id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
1,628,facial-expressions_2868585k.jpg,surprise
2,628,facial-expressions_2868584k.jpg,disgust
3,628,facial-expressions_2868582k.jpg,fear
4,dwdii,Aaron_Eckhart_0001.jpg,neutral


In [5]:
data['emotion'] = [str(i).lower() for i in data["emotion"]]

In [6]:
data['emotion']=data['emotion'].replace(to_replace='sadness', value='sad', regex=True)


In [7]:
class_labels=list(data['emotion'].unique())

In [8]:
class_labels

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sad',
 'contempt']

# USE OPEN CV2 to generate array for all images

In [9]:
import cv2


In [10]:
# Taking the column name "image" from the dataframe labelled data and converting to list
img_file_names=list(data["image"])


In [11]:
# To get the array value of each file

grayscale_image_array=[]

for i in range(len(img_file_names)):
    img_path = f"/Users/muhammadwaliji/Desktop/facial_expressions-master/images/{img_file_names[i]}"
    img = cv2.imread(img_path,0)
    grayscale_image_array.append(img)
    

### MHW Start

In [12]:
# Count the number of images for each image size

(
    pd.DataFrame.from_records(
        [img.shape for img in grayscale_image_array],
        columns=['num_rows', 'num_cols'])
    .groupby(['num_rows', 'num_cols'])
    .size()
    .rename('num_images')
    .to_frame()
    .sort_values(by="num_images", ascending=False)
)

,,num_images
num_rows,num_cols,
350,350,13265
91,73,4
69,53,3
84,63,3
85,64,3
94,73,3
37,27,3
96,70,3
98,70,2


In [13]:
grayscale_image_array_only_350by350 = []
emotions_only_350by350 = []

for i in range(len(grayscale_image_array)):
    img = grayscale_image_array[i]
    emotion = data.emotion[i]
    if img.shape == (350, 350):
        grayscale_image_array_only_350by350.append(img)
        emotions_only_350by350.append(emotion)

[array([[  9,   9,   9, ...,  42,  41,  41],
        [  9,   9,   9, ...,  41,  41,  41],
        [  8,   8,   9, ...,  42,  41,  41],
        ...,
        [160, 160, 161, ...,  68,  69,  70],
        [159, 159, 160, ...,  67,  69,  69],
        [159, 159, 160, ...,  67,  69,  69]], dtype=uint8),
 array([[10, 10, 10, ..., 92, 93, 93],
        [10, 10, 10, ..., 91, 92, 93],
        [10, 10, 11, ..., 90, 91, 91],
        ...,
        [59, 58, 54, ..., 36, 36, 36],
        [61, 60, 56, ..., 38, 39, 39],
        [61, 60, 56, ..., 38, 39, 39]], dtype=uint8),
 array([[ 92,  92,  92, ..., 107, 107, 107],
        [ 92,  92,  92, ..., 107, 107, 107],
        [ 92,  92,  92, ..., 106, 106, 106],
        ...,
        [101, 101, 102, ..., 128, 129, 129],
        [101, 101, 102, ..., 129, 130, 130],
        [101, 101, 102, ..., 129, 130, 130]], dtype=uint8),
 array([[125, 125, 118, ..., 162, 166, 166],
        [124, 124, 117, ..., 161, 165, 166],
        [121, 121, 114, ..., 158, 162, 163],
       

In [14]:
len(grayscale_image_array)

14190

In [15]:
len(grayscale_image_array_only_350by350)

13265

In [16]:
len(emotions_only_350by350)

13265

In [18]:
grayscale_image_array_only_350by350[0]

array([[  9,   9,   9, ...,  42,  41,  41],
       [  9,   9,   9, ...,  41,  41,  41],
       [  8,   8,   9, ...,  42,  41,  41],
       ...,
       [160, 160, 161, ...,  68,  69,  70],
       [159, 159, 160, ...,  67,  69,  69],
       [159, 159, 160, ...,  67,  69,  69]], dtype=uint8)

In [19]:
X_mhw = np.concatenate([
    img.reshape(1, -1)
    for img in
    grayscale_image_array_only_350by350
])

In [20]:
X_mhw.shape

(13265, 122500)

In [21]:
350 * 350

122500

### MHW End

In [ ]:
# Sheru old
# X = grayscale_image_array
# y = np.array(data["emotion"])

In [22]:
X = X_mhw
y = np.array(emotions_only_350by350)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y)

In [24]:
X_train

array([[  1,   1,   1, ...,  23,  22,  22],
       [ 13,  13,  12, ..., 120, 122, 122],
       [ 11,  10,   9, ...,  14,  14,  14],
       ...,
       [ 13,  13,  14, ...,  48,  48,  48],
       [ 48,  47,  36, ...,  13,  13,  13],
       [111, 111, 111, ...,   1,   1,   1]], dtype=uint8)

In [25]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)    

In [ ]:
model.fit(X_train,
          y_train,
          batch_size=32,
          epochs=3,
          validation_data=(X_test,y_test),
          shuffle=True
)


In [ ]:
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")


In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np



# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("frog.png", target_size=(32, 32))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

In [ ]:
# 